# Aim

This will generate the .table file for the simulations

In [23]:
import json
import numpy as np
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from bisect import bisect_left

import sys, os
sys.path.append(".")
current_path = os.getcwd()

pi = 3.14159265

In [24]:
young   = [2.7e6 - 1.1e6, 2.7e6, 2.7e6 + 1.1e6]
density = [1100 , (1100 + 2300)/2 , 2300]
root_radius = [0.9e-3, 1e-3,   1.1e-3]
radii_diff  = [0.6e-3, 0.5e-3, 0.4e-3]


params = [
    [young[1], density[1], root_radius[1], radii_diff[1]],
    [young[0], density[1], root_radius[1], radii_diff[1]],
    [young[2], density[1], root_radius[1], radii_diff[1]],
    [young[1], density[0], root_radius[1], radii_diff[1]],
    [young[1], density[2], root_radius[1], radii_diff[1]],
    [young[1], density[1], root_radius[0], radii_diff[0]],
    [young[1], density[1], root_radius[2], radii_diff[2]]
]

variation_number = len(params)

# Get number of experiments

In [25]:
# --------------------------------------------------------------------------- setting the directory
os.chdir(current_path)

# --------------------------------------------------------------------------- selecting the data files
import pandas as pd
data = pd.read_csv('exp_details.csv')
data.head() # to display the first 5 lines of loaded data


,ID,file_name,x_pos,z_pos,brush_orien
0,1,'Exp_40_1_C001H001S0001',0.0277,0.0645,3.14159265/2
1,2,'Exp_40_2_C001H001S0001',0.0277,0.0645,3.14159265/2
2,3,'Exp_40_3_C001H001S0001',0.0257,0.0645,3.14159265/2
3,4,'Exp_40_4_C001H001S0001',0.0257,0.0645,3.14159265/2
4,5,'Exp_40_5_C001H001S0001',0.0237,0.0645,3.14159265/2


## Generating the table file

In [26]:
os.chdir(current_path)

file    = open('parametric.table', "w")
number  = 12
covar   = 1e-5 

# writing the column names
# ID file_name x_pos z_pos brush_orien
file.write('ID') 
file.write(" ")

file.write('file_name') 
file.write(" ")

file.write('x_pos') 
file.write(" ")

file.write('z_pos') 
file.write(" ")

file.write('brush_orien') 
file.write(" ")

file.write('young') 
file.write(" ")

file.write('density') 
file.write(" ")

file.write('root_radius') 
file.write(" ")

file.write('radii_diff') 
file.write(" ")
file.write("\n")

for idx, i in enumerate(data['file_name']):
    for jdx, j in enumerate(params):
        file.write(str(idx * variation_number + jdx + 1))
        file.write(" ")

        file.write(str(i))
        file.write(" ")

        file.write(str(data['x_pos'][idx]))
        file.write(" ")

        file.write(str(data['z_pos'][idx]))
        file.write(" ")

        file.write(data['brush_orien'][idx])
        file.write(" ")

        for par in j:
            file.write(" ")
            file.write(str(par))

        file.write("\n")

file.close()



# For multiple VMS 
Turns out azure does not have VMs with 16 cores. So settle with 4 cores. So we have to split the runs into batches of 16 and run it on azure, HP and lenovo

In [27]:
# made an array will all the runs
run_details_str_list = []
for idx, i in enumerate(data['file_name']):
    for jdx, j in enumerate(params):
        run_string = ''
        run_string = run_string + str(idx * variation_number + jdx + 1)
        run_string = run_string + " "

        run_string = run_string + str(i)
        run_string = run_string + " "

        run_string = run_string + str(data['x_pos'][idx])
        run_string = run_string + " "

        run_string = run_string + str(data['z_pos'][idx])
        run_string = run_string + " "

        run_string = run_string + data['brush_orien'][idx]
        run_string = run_string + " "

        for par in j:
            run_string = run_string + " "
            run_string = run_string + str(par)

        run_string = run_string + "\n"

        run_details_str_list.append(run_string)


In [28]:
from math import *

number_cores = 4
hours = 4
number_run_per_VM = number_cores * hours
number_runs = len(data['file_name']) * len(params)
number_VMs  = ceil(number_runs / number_run_per_VM)

run_id_idx = 0
runs_left  = number_runs

for VM in range(number_VMs):
    with open('VM_ID_' + str(VM) + '_parametric.table', "w") as file:
        number  = 12
        covar   = 1e-5 

        # writing the column names
        # ID file_name x_pos z_pos brush_orien
        file.write('ID') 
        file.write(" ")

        file.write('file_name') 
        file.write(" ")

        file.write('x_pos') 
        file.write(" ")

        file.write('z_pos') 
        file.write(" ")

        file.write('brush_orien') 
        file.write(" ")

        file.write('young') 
        file.write(" ")

        file.write('density') 
        file.write(" ")

        file.write('root_radius') 
        file.write(" ")

        file.write('radii_diff') 
        file.write(" ")
        file.write("\n")


        no_runs_in_VM = number_run_per_VM if runs_left > number_run_per_VM else runs_left
        for run in range(no_runs_in_VM):
            file.write(run_details_str_list[run_id_idx])
            run_id_idx += 1


        runs_left = runs_left - number_run_per_VM

